# Overview

As a data scientist, working on the project HTM-application, we would like to detect anomalies for cpu usage data from smaug cluster. 
The following notebook is used to fetch the info about `openshift:cpu_usage_cores:sum` for smaug cluster. 

In [9]:
import os
import datetime as dt
from tqdm import tqdm
import pandas as pd
import warnings
import urllib3
from dotenv import load_dotenv, find_dotenv

from prometheus_api_client import PrometheusConnect, MetricSnapshotDataFrame

In [2]:
urllib3.disable_warnings()
pd.set_option("display.max_columns", None)

In [3]:
%matplotlib inline
load_dotenv(find_dotenv())
warnings.filterwarnings("ignore")

# Connect to Prometheus

We refer to the [docs](https://github.com/operate-first/apps/tree/master/docs/content/observatorium/thanos) for accesing the [Thanos](https://thanos-query-frontend-opf-observatorium.apps.smaug.na.operate-first.cloud/graph?g0.expr=&g0.tab=1&g0.stacked=0&g0.range_input=1h&g0.max_source_resolution=0s&g0.deduplicate=1&g0.partial_response=0&g0.store_matches=%5B%5D) metrics using the Prometheus Api client.

In [4]:
prom_url = os.getenv("PROM_URL")
prom_access_token = os.getenv("PROM_ACCESS_TOKEN")

PROM_URL = prom_url
PROM_ACCESS_TOKEN = prom_access_token

In [5]:
# instantiate prometheus connector
pc = PrometheusConnect(
    url=PROM_URL,
    headers={"Authorization": f"bearer {PROM_ACCESS_TOKEN}"},
    disable_ssl=True,
)

## Setting up the date

Here we set-up the date for accessing the metrics. 

In [6]:
start_time = dt.datetime(year=2021, month=10, day=1)
end_time = dt.datetime(year=2021, month=11, day=30)
step = dt.timedelta(minutes=5, seconds=0)

In [7]:
num_steps = (end_time - start_time).total_seconds() / step.total_seconds()
print(num_steps)

df_cpu = pd.DataFrame()

for i in tqdm(range(int(num_steps))):
    # get data from current time step
    ts = (start_time + i * step).timestamp()
    df = MetricSnapshotDataFrame(
        pc.custom_query(
            "openshift:cpu_usage_cores:sum{}",
            params={"time": ts},
        )
    )
    # save it
    df_cpu = df_cpu.append(df)

df_cpu.head()

17280.0


100%|██████████| 17280/17280 [08:24<00:00, 34.26it/s]


,__name__,cluster,prometheus,receive,tenant_id,timestamp,value
0,openshift:cpu_usage_cores:sum,moc/smaug,openshift-monitoring/k8s,true,moc.smaug,1637256300,173.19871643299905
0,openshift:cpu_usage_cores:sum,moc/smaug,openshift-monitoring/k8s,true,moc.smaug,1637256600,157.4071101149342
0,openshift:cpu_usage_cores:sum,moc/smaug,openshift-monitoring/k8s,true,moc.smaug,1637256900,154.68969539644834
0,openshift:cpu_usage_cores:sum,moc/smaug,openshift-monitoring/k8s,true,moc.smaug,1637257200,155.95856012168218
0,openshift:cpu_usage_cores:sum,moc/smaug,openshift-monitoring/k8s,true,moc.smaug,1637257500,152.46881932675063


# Saving the accessed dataset

In [8]:
df_cpu.to_parquet("../../data_cpu/df_cpu.parquet")

# Conclusion

We have now fetched the given metrics from the Thanos and using Prometheus API client. In the next step, we will be using the data to detect anomalies using HTM algorithm. 